In [1]:
import sqlite3

In [3]:
import jira

In [ ]:
def get_jiras():
    jira = JIRA('https://gojira.yourcompany.com', basic_auth=('username', 'password'))
    issues = jira.search_issues('labels in (Vulnerability)', maxResults=200)
    return issues

In [ ]:
def dbconnector():
    conn = sqlite3.connect('jiras.db')
    try:
        conn.execute('''CREATE TABLE jiras (hash text, id text, status text, summary text, squad text, tribe text,reporter text,vuln_category text, vuln_source text, level text, created DATETIME, closed DATETIME,overdue text)''')
        conn.commit()
    except:
        pass
    conn.close()

In [ ]:
def db_updater(issues):
    conn = sqlite3.connect('jiras.db')
    c = conn.cursor()
    for x in issues:
        hashy =hashlib.md5()
        hashy.update((x.fields.summary.encode('utf8')))
        hashy.digest()
        c.execute("SELECT hash from jiras where hash =?",(hashy.hexdigest(),))
        res = c.fetchall()
        
        if res == []:
            prjname = x.fields.project.key
            tribe=get_tribe_jira(str(prjname))
            if 'Overdue' in x.fields.labels:
                overdue = 'True'
            else:
                overdue = 'False'
            reporter = unicode(x.fields.reporter)
            c.execute("INSERT INTO jiras VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)",(str(hashy.hexdigest()),x.id,str(x.fields.status).encode('utf-8'),str(x.fields.summary.encode('ascii','ignore')),str(prjname),str(tribe),reporter,str(x.fields.customfield_19502),str(x.fields.customfield_19505),str(x.fields.priority),str(x.fields.created)[0:10],str(x.fields.resolutiondate)[0:10],overdue))
            conn.commit()
    conn.close()

In [ ]:
dbconnector()

In [ ]:
issues = get_jiras()

In [ ]:
db_updater(issues)